### Домашнее задание

Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/GB_docker_flask_example)

По шагам:
0. выбрать себе датасет (который интересен или нравится больше всего), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
1. установить удобную для себя среду разработки (pycharm прекрасен - https://www.jetbrains.com/pycharm/)
2. для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта конечно же при этом)
3. завести себе аккаунт на github (если его еще нет). У самого github есть такой "hello world" по работе с ним - https://guides.github.com/activities/hello-world/
4. итоговый проект должен содержать: 1) каталог app/models/ (здесь модель-пайплайн предобученная) 2) файл app/run_server.py (здесь основной код flask-приложения) 3) requirements.txt (список пакетов, которые у вас используются в проекте - в корне проекта) 4) README.md (здесь какое-то описание, что вы делаете, что за данные, как запускать и т.д) 5) Dockerfile 6) docker-entrypoint.sh
5. (<b>Опционально</b>): front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)

Полезные ссылки:
1. датасеты (для полета мысли): https://www.kaggle.com/datasets
2. конкурс Сбербанка по недвижимости (можно этот набор данных также взять и обучить модель предсказывать стоимость жилья - неплохой такой сервис может получиться) - https://www.kaggle.com/c/sberbank-russian-housing-market/data Там же и ноутбуки с разными подходами есть.
3. минималистичный пример связки keras/flask https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html для определения класса картинки
4. неплохой такой пример (помимо того, что разобрали на занятии) связки docker/flask - https://cloud.croc.ru/blog/byt-v-teme/flask-prilozheniya-v-docker/
5. https://www.digitalocean.com/community/tutorials/how-to-build-and-deploy-a-flask-application-using-docker-on-ubuntu-18-04

p.s. если проблемы с выбором датасета, то пишите пожалуйста - будем вместе думать)

# 1. Создаем датасет

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib qt

In [2]:
# читаем исходные данные для датасета и созадем датасет
files = ["AI.csv", "wsbp.csv", "wsrh.csv", "wsth.csv"]
i = 0

for file in files:        
    if i == 0:        
        dset = pd.read_csv(file, names = ["Time", file], skiprows=[0])
        i += 1
    else:
        dset = dset.merge(pd.read_csv(file, usecols = ["Value"]), left_index=True, right_index=True)        
        dset.rename(columns = {"Value" : file}, inplace = True)
# сохраним датасет
dset.to_csv("original_dataset.csv")
dset

,Time,AI.csv,wsbp.csv,wsrh.csv,wsth.csv
0,1/30/2021 19:02:15,37.000000,1005.968994,82.079987,-13.818200
1,1/30/2021 20:02:15,33.223209,1006.039978,80.348824,-14.015130
2,1/30/2021 21:02:15,36.000000,1006.090027,80.495888,-13.943720
3,1/30/2021 22:02:15,34.903080,1006.140015,81.833351,-14.217760
4,1/30/2021 23:02:15,34.484661,1006.190002,79.476692,-15.350870
...,...,...,...,...,...
8780,1/31/2022 15:02:15,43.375000,992.799988,86.986160,-7.212864
8781,1/31/2022 16:02:15,41.029411,992.880371,79.485641,-7.497532
8782,1/31/2022 17:02:15,41.365849,993.257019,78.728760,-7.782199
8783,1/31/2022 18:02:15,44.734810,993.633484,79.296791,-8.110570


# 2. Подготавлием данные для обучения

In [3]:
# анализируем данные. видим пропуски
dset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8785 entries, 0 to 8784
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Time      8785 non-null   object 
 1   AI.csv    8664 non-null   float64
 2   wsbp.csv  8667 non-null   float64
 3   wsrh.csv  8665 non-null   float64
 4   wsth.csv  8665 non-null   float64
dtypes: float64(4), object(1)
memory usage: 343.3+ KB


In [4]:
# преобразуем время в формат времени из строк
dset["Time"] = pd.to_datetime(dset["Time"]) 

In [5]:
# заполняем пропуски
def fill_nan(data):    
        mean = data.mean()
#         print(data.name,data.mean())        
        data.fillna(mean, inplace=True)    

In [6]:
for column in files:
    fill_nan(dset.loc[:,column])

In [7]:
# посмотрим распределение величин
# fig, axs = plt.subplots(1, 5, figsize=(12, 7))
for feature in dset.columns:
    sns.displot(x=feature, data=dset);

In [8]:
# зададим доп колонку для месяца по времени
dset["month"] = 0
dset["month"] = dset["Time"].dt.month

In [9]:
# зададим доп колонку для дня
dset["day"] = 0
dset["day"] = dset["Time"].dt.day

In [10]:
# конвертируем категориальные признаки в бинарные
def convertdummies(X, columns):
    dset_new = X.copy() # датасет с категориальными признаками
    for column in columns:
        dset_temp = pd.get_dummies(X[column], prefix=column)
        dset_new = pd.merge(dset_new, dset_temp, left_index=True, right_index=True)
        dset_new.drop(column, axis=1, inplace=True) # удаляем исходные 
    columns_new = dset_new.columns
    return dset_new, columns_new

In [22]:
# конвертируем категориальные признаки в бинарные
columns_categorical = ["month", "day"]
dset_new, columns_new = convertdummies(dset, columns_categorical)

# 3. Обучаем регрессию

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

## 3.1 чистые данные

In [25]:
# задаем target
y = dset.iloc[:,1]

# задаем features
X = dset.copy()
X = X.drop(["AI.csv", "Time"], axis = 1)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [27]:
result = []

In [28]:
lr = LinearRegression()
lr.fit(X_train, y_train)
preds = lr.predict(X_test)
result.append({"R2_score": r2_score(preds, y_test),
               "Type": "Чистая регрессия"
              })

In [29]:
# посмотрим веса признаков
features = pd.DataFrame(lr.coef_[:3], 
                        files[1:], 
                        columns=['coefficient'])
features

,coefficient
wsbp.csv,0.010959
wsrh.csv,0.324206
wsth.csv,1.224347


## 3.2 Скалированные данные

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
# стандартизуем входные данные
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)

X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

In [32]:
lr_scaled = LinearRegression()
lr_scaled.fit(X_train_scaled, y_train)
preds_scaled = lr_scaled.predict(X_test_scaled)
result.append({"R2_score": r2_score(preds_scaled, y_test),
               "Type": "скалированная регрессия"
              })

In [33]:
result

[{'R2_score': 0.7562792801797547, 'Type': 'Чистая регрессия'},
 {'R2_score': 0.7562792801797545, 'Type': 'скалированная регрессия'}]

Почему то результат одинаковый с не стандартизованными данными, хотя масштабы отличаются на порядок - два.

## 3.3 Добавляем дополнительные features

In [34]:
y_new = dset_new.iloc[:,1]
X_new = dset_new.copy()
X_new = X_new.drop(["AI.csv", "Time"], axis = 1)

In [35]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, random_state=0)

In [36]:
lr_new = LinearRegression()
lr_new.fit(X_train_new, y_train_new)
preds_new = lr_new.predict(X_test_new)
result.append({"R2_score": r2_score(preds_new, y_test_new),
               "Type": "регрессия c доп фичами"
              })

In [37]:
result

[{'R2_score': 0.7562792801797547, 'Type': 'Чистая регрессия'},
 {'R2_score': 0.7562792801797545, 'Type': 'скалированная регрессия'},
 {'R2_score': 0.8590059532520475, 'Type': 'регрессия c доп фичами'}]

Результат гораздо выше чем без категориального признака месяца

# 4. Обучаем бустинг

In [38]:
# Попробуем бустинг
# обязательно удостовериться что версия библиотек одинаковая в conda и среде запуска серверной части иначе будет ошибка
# %pip install --upgrade xgboost
import xgboost as xgb

## 4.1 Без дополнительных features

In [39]:
xgboost = xgb.XGBRegressor()
xgboost.fit(X_train, y_train)
xgb_predict = xgboost.predict(X_test)
result.append({"R2_score": r2_score(xgb_predict, y_test),
               "Type": "бустинг без доп фич"
              })

## 4.2 C дополнительными features

In [40]:
xgboost_new = xgb.XGBRegressor()
xgboost_new.fit(X_train_new, y_train)
xgb_predict_new = xgboost_new.predict(X_test_new)
result.append({"R2_score": r2_score(xgb_predict_new, y_test),
               "Type": "бустинг c доп фичами"
              })

In [41]:
result

[{'R2_score': 0.7562792801797547, 'Type': 'Чистая регрессия'},
 {'R2_score': 0.7562792801797545, 'Type': 'скалированная регрессия'},
 {'R2_score': 0.8590059532520475, 'Type': 'регрессия c доп фичами'},
 {'R2_score': 0.9378978480775644, 'Type': 'бустинг без доп фич'},
 {'R2_score': 0.9388382751165215, 'Type': 'бустинг c доп фичами'}]

In [42]:
# сохраним test
X_test_new.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
# сохраним train
X_train_new.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [43]:
import dill
# сохраняем модель
with open("xgboost_model.dill", "wb") as f:
    dill.dump(xgboost_new, f)

In [46]:
col_month = ["month_" + str(i)  for i in range(1, 13)] # column names into Test dataset
col_day = ["day_" + str(i) for i in range(1, 32)]  # column names into Test dataset
df_months = pd.DataFrame(0, index=range(1), columns=col_month + col_day)
df_months

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,...,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Бустинг с доп фичами дал наилучший результат. Его и будем использовать